#### **PART 01 Data Loading:**

**The Work Mythology in Part 01 Data Loading and Intial Cleaning:**

>**Steps:**
>1. Import Packages
>2. Ensuring Folder Structure, a main file named data, and subfolders: Raw and Clean.
>3. Download files from the Web into a raw file in the home repository
>4. Additional Data loading, such as converting and unzipping a  file
>5. Identifying which columns to keep from raw file structure
>6. Build CLEAN versions in data/clean (from RAW)
>7.  Load from data/clean with selected columns
>8.  Clean column names function that does the initial cleaning on the data before in-depth analysis

In [8]:
# Step 01 - Import packages
import os
import urllib.request
import zipfile
import pandas as pd
import geopandas as gpd

In [12]:
# Step 02 - Ensuring Folder Structure: Create a main file named data and subfolders named raw and clean.
def ensure_data_folders():
    folders = ["data/raw", "data/clean"]
    
    for folder in folders:
        if not os.path.isdir(folder):
            os.makedirs(folder)
            print(f"Created folder: {folder}")
        else:
            print(f"Folder already exists: {folder}")

    print("--" * 30)

ensure_data_folders()

Folder already exists: data/raw
Folder already exists: data/clean
------------------------------------------------------------


In [13]:
# Step 03 - Download files from the Web into raw file on home repositry
def download_if_missing(url, local_name):
    dest = os.path.join("data", "raw", local_name)
    
    print(f"Let me check: {local_name}")

    if os.path.isfile(dest):
        print("File already exists — skipping download.")
    else:
        print(f"Downloading from:\n    {url}")
        urllib.request.urlretrieve(url, dest)
        print("Download complete — saved.")

    print("--" * 20)
    return dest 

# Base File data folder attributed by CASA
base = "https://orca.casa.ucl.ac.uk/~jreades/data/"

orca_files = [
    "20250615-London-listings.csv.gz",
    "MSOA-2011.gpkg"
]

# Extra files we saw necessary to add for further analysis
extra_files = {
    "housing_msoa.csv":
        "https://data.london.gov.uk/download/2z0yn/20264159-36cb-4aa2-8371-ae884ae83e88/msoa-data.csv",

    "local-authority-rents-borough.xlsx":
        "https://data.london.gov.uk/download/2g1k1/2a74e893-3188-4cb8-b5a7-087a1965b623/local-authority-rents-borough.xlsx",

    "london_boroughs.zip":
        "https://data.london.gov.uk/download/20od9/08d31995-dd27-423c-a987-57fe8e952990/London-wards-2018.zip",
    
    "council_tax_borough.xlsx":
        "https://data.london.gov.uk/download/expnl/59cc7c37-da8f-4158-bc47-491c3d167b05/council-tax-bands-borough.xlsx"
}

local_paths = []

print("!STARTING ORCA DOWNLOADS!")

for fname in orca_files:
    url = base + fname
    path = download_if_missing(url, fname)
    local_paths.append(path)

print("!STARTING DATASTORE DOWNLOADS!")

for local_name, url in extra_files.items():
    path = download_if_missing(url, local_name)
    local_paths.append(path)

!STARTING ORCA DOWNLOADS!
Let me check: 20250615-London-listings.csv.gz
File already exists — skipping download.
----------------------------------------
Let me check: MSOA-2011.gpkg
File already exists — skipping download.
----------------------------------------
!STARTING DATASTORE DOWNLOADS!
Let me check: housing_msoa.csv
File already exists — skipping download.
----------------------------------------
Let me check: local-authority-rents-borough.xlsx
File already exists — skipping download.
----------------------------------------
Let me check: london_boroughs.zip
File already exists — skipping download.
----------------------------------------
Let me check: council_tax_borough.xlsx
File already exists — skipping download.
----------------------------------------


In [31]:
# Step 04 -  Additional Data loading, such as converting and unzipping a  file

# Convert Rent file from Excel to CSV
print("!CONVERTING RENT EXCEL (Local Authority Sheet) TO CSV!")

excel_path = "data/raw/local-authority-rents-borough.xlsx"
csv_path   = "data/raw/local-authority-rents-borough.csv"

if os.path.isfile(csv_path):
    print(f"CSV already exists, skipping conversion: {csv_path}")
else:
    if not os.path.isfile(excel_path):
        print(f"Excel file not found: {excel_path}")
    else:
        rents_xlsx = pd.read_excel(
            excel_path,
            sheet_name="Local Authority Rents"
        )
        rents_xlsx.to_csv(csv_path, index=False)
        print(f"Converted Excel to CSV: {csv_path}")

print("--" * 20)


# Convert Council Tax file from Excel to CSV
print("!CONVERTING COUNCIL TAX EXCEL (2024-2025) TO CSV!")

excel_path = "data/raw/council_tax_borough.xlsx"
csv_path   = "data/raw/council_tax_borough.csv"

if os.path.isfile(csv_path):
    print(f"CSV already exists, skipping conversion: {csv_path}")
else:
    if not os.path.isfile(excel_path):
        print(f"Excel file not found: {excel_path}")
    else:
        tax_xlsx = pd.read_excel(
            excel_path,
            sheet_name="2024-25"
        )
        tax_xlsx.to_csv(csv_path, index=False)
        print(f"Converted Excel to CSV: {csv_path}")

print("--" * 20)

print("!UNZIPPING LONDON BOROUGHS SHAPEFILE!")

# unzipping Shape File of London Borough
zip_path = "data/raw/london_boroughs.zip"
if os.path.isfile(zip_path):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("data/raw/")
    print("Unzipped london_boroughs.zip")
else:
    print("Zip file not found: data/raw/london_boroughs.zip")

print("--" * 20)

!CONVERTING RENT EXCEL (Local Authority Sheet) TO CSV!
CSV already exists, skipping conversion: data/raw/local-authority-rents-borough.csv
----------------------------------------
!CONVERTING COUNCIL TAX EXCEL (2024-2025) TO CSV!
CSV already exists, skipping conversion: data/raw/council_tax_borough.csv
----------------------------------------
!UNZIPPING LONDON BOROUGHS SHAPEFILE!
Unzipped london_boroughs.zip
----------------------------------------


In [32]:
# Step 05 - Identifying which columns to keep from raw file structure

#Listings 
cols_listings = [
    'id', 'listing_url', 'last_scraped', 'name', 
    'description', 'host_id', 'host_since', 'host_is_superhost', 
    'host_listings_count', 'host_total_listings_count',
    'neighbourhood_cleansed', 'latitude', 'longitude', 
    'property_type', 'room_type', 'accommodates', 
    'price', 'minimum_nights', 'maximum_nights', 
    'minimum_minimum_nights','maximum_minimum_nights',
    'minimum_maximum_nights','maximum_maximum_nights',
    'minimum_nights_avg_ntm','maximum_nights_avg_ntm',
    'availability_365',
    'number_of_reviews', 
    'first_review', 'last_review',
    'review_scores_rating', 
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms',
    'reviews_per_month',
    'estimated_revenue_l365d',
    'estimated_occupancy_l365d'
]

# Housing (2011 only)
cols_housing = ["Middle Super Output Area",
                 "MSOA Name",
                 "Households (2011);All Households;"
                 ]

# Rent: only latest year
cols_rent = ["Code",
              "New Code",
              "Area",
              "2023-24"
             ]

# Council tax: band D comparison as it is the Band used as reference by the GLA !!! ADD REF
cols_tax = [
    "Code",
    "Local authority",
    "Band D"
]

In [34]:
# Step 06 -  Build CLEAN versions in data/clean (from RAW)
# Each File goes through the raw version and into clean which would allow us to manipulate the data without losing the raw

print("!CREATING CLEAN VERSIONS!")

# Listings
raw_listings_path = "data/raw/20250615-London-listings.csv.gz"
clean_listings_path = "data/clean/20250615-London-listings.csv.gz"

if not os.path.isfile(clean_listings_path):
    listings_raw = pd.read_csv(
        raw_listings_path,
        compression="gzip",
        encoding="latin1",
        low_memory=False
    )
    listings = listings_raw[cols_listings]
    listings.to_csv(
        clean_listings_path,
        index=False,
        compression="gzip"
    )
    print("Saved cleaned listings to data/clean")
else:
    print("Clean listings already exist, skipping.")

# MSOA
raw_msoa_path = "data/raw/MSOA-2011.gpkg"
clean_msoa_path = "data/clean/MSOA-2011.gpkg"

if not os.path.isfile(clean_msoa_path):
    msoa = gpd.read_file(raw_msoa_path)
    msoa.to_file(clean_msoa_path, driver="GPKG")
    print("Saved MSOA to data/clean")
else:
    print("Clean MSOA already exists, skipping.")

# Rent (already converted to CSV in raw)
raw_rent_path = "data/raw/local-authority-rents-borough.csv"
clean_rent_path = "data/clean/local-authority-rents-borough.csv"

if not os.path.isfile(clean_rent_path):
    rent_raw = pd.read_csv(raw_rent_path, encoding="latin1", low_memory=False)
    rent = rent_raw[cols_rent]
    rent.to_csv(clean_rent_path, index=False)
    print("Saved cleaned rent to data/clean")
else:
    print("Clean rent already exists, skipping.")

# Housing
raw_housing_path = "data/raw/housing_msoa.csv"
clean_housing_path = "data/clean/housing_msoa.csv"

if not os.path.isfile(clean_housing_path):
    housing_raw = pd.read_csv(raw_housing_path, encoding="latin1", low_memory=False)
    housing = housing_raw[cols_housing]
    housing.to_csv(clean_housing_path, index=False)
    print("Saved cleaned housing to data/clean")
else:
    print("Clean housing already exists, skipping.")

# Borough and Wards
borough_clean_dir = "data/clean/London-wards-2018_ESRI"
os.makedirs(borough_clean_dir, exist_ok=True)

# this path depends on how the zip unzips 
raw_borough_path = "data/raw/London-wards-2018_ESRI/London_Ward.shp"

clean_borough_path = os.path.join(borough_clean_dir, "London_Ward.shp")

if not os.path.isfile(clean_borough_path):
    borough = gpd.read_file(raw_borough_path)
    borough.to_file(clean_borough_path)
    print("Saved borough shapefile to data/clean")
else:
    print("Clean borough shapefile already exists, skipping.")

# Council Tax
raw_tax_path = "data/raw/council_tax_borough.csv"
clean_tax_path = "data/clean/council_tax_borough.csv"

if not os.path.isfile(clean_tax_path):
    council_tax_raw = pd.read_csv(raw_tax_path, encoding="latin1", low_memory=False)
    council_tax = council_tax_raw[cols_tax]
    council_tax.to_csv(clean_tax_path, index=False)
    print("Saved cleaned council tax to data/clean")
else:
    print("Clean council tax already exists, skipping.")

print("**All cleaned files saved to data/clean/**")


!CREATING CLEAN VERSIONS!
Clean listings already exist, skipping.
Clean MSOA already exists, skipping.
Clean rent already exists, skipping.
Clean housing already exists, skipping.
Clean borough shapefile already exists, skipping.
Clean council tax already exists, skipping.
**All cleaned files saved to data/clean/**


In [36]:
# Step 07 -  Load from data/clean with selected columns

print("!LOADING FILES (SELECTED COLUMNS ONLY)!")

# Listings
listings = pd.read_csv(
    "data/clean/20250615-London-listings.csv.gz",
    compression="gzip",
    usecols=cols_listings,
    encoding="latin1",
    low_memory=False
)
print(f"Step 01: listings loaded: {listings.shape[0]:,} rows, {listings.shape[1]} columns")

# MSOA
msoa = gpd.read_file("data/clean/MSOA-2011.gpkg")
print(f"Step 02: msoa loaded: {msoa.shape[0]:,} polygons, {msoa.shape[1]} columns")

# Rent
rent = pd.read_csv(
    "data/clean/local-authority-rents-borough.csv",
    usecols=cols_rent,
    encoding="latin1",
    low_memory=False
)
print(f"Step 03: rent loaded: {rent.shape[0]:,} rows, {rent.shape[1]} columns")

# Housing
housing = pd.read_csv(
    "data/clean/housing_msoa.csv",
    usecols=cols_housing,
    encoding="latin1",
    low_memory=False
)
print(f"Step 04: housing loaded: {housing.shape[0]:,} rows, {housing.shape[1]} columns")

# Borough boundary polygons
borough = gpd.read_file("data/clean/London-wards-2018_ESRI/London_Ward.shp")
print(f"Step 05: borough loaded: {borough.shape[0]:,} polygons, {borough.shape[1]} columns")

# Council Tax
council_tax = pd.read_csv(
    "data/clean/council_tax_borough.csv",
    usecols=cols_tax,
    encoding="latin1",
    low_memory=False
)
print(f"Step 06: council tax loaded: {council_tax.shape[0]:,} rows, {council_tax.shape[1]} columns")

print("**All selected files successfully loaded.**")

!LOADING FILES (SELECTED COLUMNS ONLY)!
Step 01: listings loaded: 96,651 rows, 37 columns
Step 02: msoa loaded: 983 polygons, 13 columns
Step 03: rent loaded: 46 rows, 4 columns
Step 04: housing loaded: 984 rows, 3 columns
Step 05: borough loaded: 657 polygons, 7 columns
Step 06: council tax loaded: 34 rows, 3 columns
**All selected files successfully loaded.**


In [37]:
# Step 08 - Clean column names function that does the initial cleaning on the data before in-depth analysis
def clean_names(df):
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace(r"[^\w]", "", regex=True)
    )
    return df

dfs = {
    "listings": listings,
    "msoa": msoa,
    "rent": rent,
    "housing": housing,
    "borough": borough,
    "council_tax": council_tax
}

for name, frame in dfs.items():
    dfs[name] = clean_names(frame)
    print(f"Cleaned column names for: {name}")

listings     = dfs["listings"]
msoa         = dfs["msoa"]
rent         = dfs["rent"]
housing      = dfs["housing"]
borough      = dfs["borough"]
council_tax  = dfs["council_tax"]


Cleaned column names for: listings
Cleaned column names for: msoa
Cleaned column names for: rent
Cleaned column names for: housing
Cleaned column names for: borough
Cleaned column names for: council_tax


In [38]:
rent.columns
# Just to make sure its clean and not manipulated

Index(['code', 'new_code', 'area', '202324'], dtype='object')

#### **Question 3: How many properties would be affected by the opposition’s proposal?** Emilia

In Our Evaluation, we want to create an extreme scenario to evaluate the magnitude of the effect of increased council tax: how many short-term rental properties owned by professional landlords would become unprofitable if council tax rates were increased. This requires linking landlord type, location, borough-level council tax, and the property’s estimated monthly profit, as well as evaluating companies with multiple listings and their profit-and-loss statements. 

**Data:**
1. Professional landlord listings (from Q2)
2. Borough boundaries (spatial join)
3. Borough-level council tax
4. Monthly profit estimates for each listing

**Output:** FIX THIS
1. Number of professional landlords
2. Distribution of their listings across boroughs
3. Current vs future council tax per borough
4. Number and percentage of listings becoming unprofitable
5. Scatter plot showing profit before and after tax increase and affected properties

**Issues:** FIX THIS
1. Borough-level council tax bands vary depending on property type; using averages simplifies reality.
2. Monthly profit estimates depend on assumptions about occupancy and nightly rates.
3. Listings with missing borough information must be dropped or cleaned.
4. Definition of “professional landlord” strongly shapes results.
5. The proposed % tax increase is simplified and may not reflect the real banding structure.